In [9]:
# Import libraries 
import pandas as pd
import numpy as np
##import warnings 
#warnings.filterwarnings('ignore')

In [10]:
# Import dataset 
df = pd.read_csv(r'E:\MLproject\powergauge\notebook\data\Homestead Electricity Consumption.csv')

df.head()

,Unnamed: 0,Date,Homestead_maxtempC,Homestead_mintempC,Homestead_DewPointC,Homestead_FeelsLikeC,Homestead_HeatIndexC,Homestead_WindChillC,Homestead_WindGustKmph,Homestead_cloudcover,Homestead_humidity,Homestead_precipMM,Homestead_pressure,Homestead_tempC,Homestead_visibility,Homestead_winddirDegree,Homestead_windspeedKmph,Consumption
0,0,2020-06-25 17:00:00,33.0,27.0,23.0,36.0,36.0,31.0,24.0,9.0,62.0,0.0,1018.0,31.0,10.0,94.0,19.0,103.0
1,1,2020-06-25 16:00:00,33.0,27.0,23.0,37.0,37.0,32.0,24.0,7.0,60.0,0.0,1018.0,32.0,10.0,96.0,20.0,99.0
2,2,2020-06-25 15:00:00,33.0,27.0,23.0,38.0,38.0,33.0,23.0,5.0,57.0,0.0,1019.0,33.0,10.0,98.0,20.0,74.0
3,3,2020-06-25 14:00:00,33.0,27.0,23.0,37.0,37.0,32.0,22.0,6.0,57.0,0.0,1019.0,32.0,10.0,96.0,19.0,82.0
4,4,2020-06-25 13:00:00,33.0,27.0,23.0,37.0,37.0,32.0,21.0,7.0,58.0,0.0,1019.0,32.0,10.0,95.0,18.0,73.0


In [11]:
# Data Summary 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22201 entries, 0 to 22200
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               22201 non-null  int64  
 1   Date                     22201 non-null  object 
 2   Homestead_maxtempC       22201 non-null  float64
 3   Homestead_mintempC       22201 non-null  float64
 4   Homestead_DewPointC      22201 non-null  float64
 5   Homestead_FeelsLikeC     22201 non-null  float64
 6   Homestead_HeatIndexC     22201 non-null  float64
 7   Homestead_WindChillC     22201 non-null  float64
 8   Homestead_WindGustKmph   22201 non-null  float64
 9   Homestead_cloudcover     22201 non-null  float64
 10  Homestead_humidity       22201 non-null  float64
 11  Homestead_precipMM       22201 non-null  float64
 12  Homestead_pressure       22201 non-null  float64
 13  Homestead_tempC          22201 non-null  float64
 14  Homestead_visibility  

In [12]:
# Drop columns 
df = df.drop(columns={'Unnamed: 0', 'Date'})
df.head()

,Homestead_maxtempC,Homestead_mintempC,Homestead_DewPointC,Homestead_FeelsLikeC,Homestead_HeatIndexC,Homestead_WindChillC,Homestead_WindGustKmph,Homestead_cloudcover,Homestead_humidity,Homestead_precipMM,Homestead_pressure,Homestead_tempC,Homestead_visibility,Homestead_winddirDegree,Homestead_windspeedKmph,Consumption
0,33.0,27.0,23.0,36.0,36.0,31.0,24.0,9.0,62.0,0.0,1018.0,31.0,10.0,94.0,19.0,103.0
1,33.0,27.0,23.0,37.0,37.0,32.0,24.0,7.0,60.0,0.0,1018.0,32.0,10.0,96.0,20.0,99.0
2,33.0,27.0,23.0,38.0,38.0,33.0,23.0,5.0,57.0,0.0,1019.0,33.0,10.0,98.0,20.0,74.0
3,33.0,27.0,23.0,37.0,37.0,32.0,22.0,6.0,57.0,0.0,1019.0,32.0,10.0,96.0,19.0,82.0
4,33.0,27.0,23.0,37.0,37.0,32.0,21.0,7.0,58.0,0.0,1019.0,32.0,10.0,95.0,18.0,73.0


In [13]:
# Rename columns 
df = df.rename(columns={'Homestead_maxtempC': 'Max Temp °C', 'Homestead_mintempC':'Min temp °C', 
                        'Homestead_DewPointC':'Dewpoint °C', 'Homestead_FeelsLikeC':'FeelsLike °C',
                        'Homestead_HeatIndexC':'Heat Index °C', 'Homestead_WindChillC':'Wind Chill °C', 
                        'Homestead_WindGustKmph':'Wind Gust Kmph', 'Homestead_cloudcover':'Cloud Cover',
                        'Homestead_humidity':'Humidity', 'Homestead_precipMM':'Precipitation MM',
                        'Homestead_pressure':'Pressure', 'Homestead_tempC':'Temp °C', 'Homestead_visibility':'Visibility',
                        'Homestead_winddirDegree':'Wind Direction Degree', 'Homestead_windspeedKmph':'Wind speed Kmph'})

df.head()

,Max Temp °C,Min temp °C,Dewpoint °C,FeelsLike °C,Heat Index °C,Wind Chill °C,Wind Gust Kmph,Cloud Cover,Humidity,Precipitation MM,Pressure,Temp °C,Visibility,Wind Direction Degree,Wind speed Kmph,Consumption
0,33.0,27.0,23.0,36.0,36.0,31.0,24.0,9.0,62.0,0.0,1018.0,31.0,10.0,94.0,19.0,103.0
1,33.0,27.0,23.0,37.0,37.0,32.0,24.0,7.0,60.0,0.0,1018.0,32.0,10.0,96.0,20.0,99.0
2,33.0,27.0,23.0,38.0,38.0,33.0,23.0,5.0,57.0,0.0,1019.0,33.0,10.0,98.0,20.0,74.0
3,33.0,27.0,23.0,37.0,37.0,32.0,22.0,6.0,57.0,0.0,1019.0,32.0,10.0,96.0,19.0,82.0
4,33.0,27.0,23.0,37.0,37.0,32.0,21.0,7.0,58.0,0.0,1019.0,32.0,10.0,95.0,18.0,73.0


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# Evaluation 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [15]:
# Define the features and target variable 

# drop null values
df = df.dropna()

# Features 
X = df.drop(columns={'Consumption'})

# Target variable 
y = df['Consumption']

In [16]:
# Training set and testing split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [17]:
# Initialize the standard scaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the testing data
X_test_scaled = scaler.transform(X_test)

In [25]:
from sklearn.model_selection import StratifiedKFold


# Define the XGBoost model
xgb_model = XGBRegressor()

# Define the number of folds for cross-validation
n_folds = 5

# Create a StratifiedKFold object
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize lists to store the evaluation metrics
mae_scores = []
r2_scores = []
mape_scores = []
mse_scores = []
rmse_scores = []

# Perform stratified k-fold cross-validation
for train_index, test_index in skf.split(X, y):
    # Split the data into training and testing sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize the standard scaler
    scaler = StandardScaler()

    # Fit and transform the training data
    X_train_scaled = scaler.fit_transform(X_train)

    # Transform the testing data
    X_test_scaled = scaler.transform(X_test)


    # Fit the model on the training data
    xgb_model.fit(X_train_scaled, y_train)

    # Make predictions on the testing data
    y_pred = xgb_model.predict(X_test_scaled)

    # Evaluate the model on the testing data
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    # Store the evaluation metrics
    mae_scores.append(mae)
    r2_scores.append(r2)
    mape_scores.append(mape)
    mse_scores.append(mse)
    rmse_scores.append(rmse)

# Calculate the average evaluation metrics
mae_avg = np.mean(mae_scores)
r2_avg = np.mean(r2_scores)
mape_avg = np.mean(mape_scores)
mse_avg = np.mean(mse_scores)
rmse_avg = np.mean(rmse_scores)

# Print the average evaluation metrics
print("MAE:", mae_avg)
print("R2:", r2_avg)
print("MAPE:", mape_avg)
print("MSE:", mse_avg)
print("RMSE:", rmse_avg)

MAE: 8.01223442059759
R2: 0.6854020439809962
MAPE: 13.206206084213225
MSE: 103.43808278316196
RMSE: 10.169962287078185


In [18]:

def model(X_train_scaled, y_train, X_test_scaled, y_test):
    # Initialize the XGBRegressor model
    xgb_model = XGBRegressor(n_estimators=30, random_state=42)

    # Fit the model on the entire training data
    xgb_model.fit(X_train_scaled, y_train)

    # Make predictions on the testing data
    y_pred = xgb_model.predict(X_test_scaled)

    # Evaluate the model on the testing data
    mae_test = mean_absolute_error(y_test, y_pred)
    r2_test = r2_score(y_test, y_pred)
    mape_test = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mse_test = mean_squared_error(y_test, y_pred)
    rmse_test = np.sqrt(mse_test)

    # Return the evaluation metrics
    return mae_test, r2_test, mape_test, mse_test, rmse_test

# Example usage:
# Assuming you have defined X_train_scaled, y_train, X_test_scaled, and y_test
mae_test, r2_test, mape_test, mse_test, rmse_test = model(X_train_scaled, y_train, X_test_scaled, y_test)

print("MAE:", mae_test)
print("R2:", r2_test)
print("MAPE:", mape_test)
print("MSE:", mse_test)
print("RMSE:", rmse_test)

MAE: 8.854473594372775
R2: 0.6348097352284057
MAPE: 14.604945759998742
MSE: 120.87602479312447
RMSE: 10.99436331913424


In [19]:
def model(X_train_scaled, y_train, X_test_scaled, y_test):
    # Initialize the XGBRegressor model
    rf_model = RandomForestRegressor()

    # Fit the model on the entire training data
    rf_model.fit(X_train_scaled, y_train)

    # Make predictions on the testing data
    y_pred = rf_model.predict(X_test_scaled)

    # Evaluate the model on the testing data
    mae_test = mean_absolute_error(y_test, y_pred)
    r2_test = r2_score(y_test, y_pred)
    mape_test = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mse_test = mean_squared_error(y_test, y_pred)
    rmse_test = np.sqrt(mse_test)

    # Return the evaluation metrics
    return mae_test, r2_test, mape_test, mse_test, rmse_test

# Example usage:
# Assuming you have defined X_train_scaled, y_train, X_test_scaled, and y_test
mae_test, r2_test, mape_test, mse_test, rmse_test = model(X_train_scaled, y_train, X_test_scaled, y_test)

print("MAE:", mae_test)
print("R2:", r2_test)
print("MAPE:", mape_test)
print("MSE:", mse_test)
print("RMSE:", rmse_test)

MAE: 7.347431093496667
R2: 0.7271726711754497
MAPE: 12.150934498611557
MSE: 90.30438690326068
RMSE: 9.502862037473799


In [21]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [22]:
def model(X_train_scaled, y_train, X_test_scaled, y_test):
    # Initialize the XGBRegressor model
    et_model = ExtraTreesRegressor()

    # Fit the model on the entire training data
    et_model.fit(X_train_scaled, y_train)

    # Make predictions on the testing data
    y_pred = et_model.predict(X_test_scaled)

    # Evaluate the model on the testing data
    mae_test = mean_absolute_error(y_test, y_pred)
    r2_test = r2_score(y_test, y_pred)
    mape_test = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mse_test = mean_squared_error(y_test, y_pred)
    rmse_test = np.sqrt(mse_test)

    # Return the evaluation metrics
    return mae_test, r2_test, mape_test, mse_test, rmse_test

# Example usage:
# Assuming you have defined X_train_scaled, y_train, X_test_scaled, and y_test
mae_test, r2_test, mape_test, mse_test, rmse_test = model(X_train_scaled, y_train, X_test_scaled, y_test)

print("MAE:", mae_test)
print("R2:", r2_test)
print("MAPE:", mape_test)
print("MSE:", mse_test)
print("RMSE:", rmse_test)

MAE: 6.799192938209331
R2: 0.7560882659409939
MAPE: 11.235022412076367
MSE: 80.73347966132229
RMSE: 8.985181114553134


In [24]:
from sklearn.model_selection import StratifiedKFold

# Initialize the XGBRegressor model
et_model = ExtraTreesRegressor()

# Define the number of folds for cross-validation
n_folds = 5

# Create a StratifiedKFold object
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize lists to store the evaluation metrics
mae_scores = []
r2_scores = []
mape_scores = []
mse_scores = []
rmse_scores = []

# Perform stratified k-fold cross-validation
for train_index, test_index in skf.split(X, y):
    # Split the data into training and testing sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


    # Initialize the standard scaler
    scaler = StandardScaler()

    # Fit and transform the training data
    X_train_scaled = scaler.fit_transform(X_train)

    # Transform the testing data
    X_test_scaled = scaler.transform(X_test)


    # Fit the model on the training data
    et_model.fit(X_train_scaled, y_train)

    # Make predictions on the testing data
    y_pred = et_model.predict(X_test_scaled)

    # Evaluate the model on the testing data
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    # Store the evaluation metrics
    mae_scores.append(mae)
    r2_scores.append(r2)
    mape_scores.append(mape)
    mse_scores.append(mse)
    rmse_scores.append(rmse)

# Calculate the average evaluation metrics
mae_avg = np.mean(mae_scores)
r2_avg = np.mean(r2_scores)
mape_avg = np.mean(mape_scores)
mse_avg = np.mean(mse_scores)
rmse_avg = np.mean(rmse_scores)

# Print the average evaluation metrics
print("MAE:", mae_avg)
print("R2:", r2_avg)
print("MAPE:", mape_avg)
print("MSE:", mse_avg)
print("RMSE:", rmse_avg)

MAE: 6.746451810879783
R2: 0.75649427917282
MAPE: 11.141870865995454
MSE: 80.06279018643258
RMSE: 8.947409056935156
